In [ ]:
import scanpy as sc
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Open downsampled oral single cell dataset
adata = sc.read_h5ad("path_to_sc_dataset.h5ad")
#Create a subset of the adata to only include cells from the Williams et all dataset
adata_Wil = adata[adata.obs['ref'] == 'Williams 2021'].copy()
#Subset to include only epithelial cells
adata_Wil_Epi = adata_Wil[adata_Wil.obs['generalCellTypes'] == 'Epithelial'].copy()
#Subset to only include gingival tissue cells
adata_Wil_GEpi = adata_Wil_Epi[adata_Wil_Epi.obs['tissue'] == 'gingiva'].copy()

In [ ]:
#Open the Xenium dataset
ref_Xen = sc.read_h5ad("path_to_Xenium_dataset")
#Subset to include only epithelial cells
ref_Xen_Epi = ref_Xen[ref_Xen.obs['Lvl3'].isin(["Ep.TA", "Ep.or.b-pb", "Ep.or.sp", "Ep.or.k"])].copy()


In [ ]:
#Concatenate datasets
assay_data = pd.Series('scSeq', index=adata_Wil_GEpi.obs.index)
adata_Wil_GEpi.obs['assay'] = assay_data

merged = adata_Wil_GEpi.concatenate(
    ref_Xen_Epi, batch_key="assay", batch_categories=["scSeq", "xenium"]
)

In [ ]:
sc.pp.scale(merged)
sc.tl.pca(merged)

In [ ]:
# This step takes time; ~10 min per iteration
sc.external.pp.harmony_integrate(merged, key="assay", max_iter_harmony=20, max_iter_kmeans=30)

In [ ]:
# Visualize merged with UMAP embedding; takes a long time
sc.pp.neighbors(merged, n_neighbors=50, use_rep="X_pca_harmony", metric="correlation")
sc.tl.umap(merged, min_dist=0.5)

In [ ]:
sc.pl.umap(merged, color='assay')

In [ ]:
# Transfer annotations from xenium to scRNAseq
nn = KNeighborsClassifier(n_neighbors=1, n_jobs=16, weights='distance', metric='euclidean')
train = merged[merged.obs["assay"] == "xenium"]
nn.fit(train.obsm["X_pca_harmony"], train.obs['Lvl3']) 
labels = nn.predict(merged[merged.obs["assay"] == "scSeq"].obsm["X_pca_harmony"])
merged.obs["xenium_to_sc_label"] = pd.Series(labels, index=merged[merged.obs["assay"] == "scSeq"].obs.index)

In [ ]:
merged_sc = merged[merged.obs['assay'] == 'scSeq'].copy()

In [ ]:
# Visualize xenium annotations vs transferred scRNAseq annotations
xen_obj = merged[merged.obs['assay']=='scSeq']
xen_obj.obs['clusterCellTypes'] = xen_obj.obs['clusterCellTypes'].astype('str')
celltype_counts = pd.DataFrame(xen_obj.obs.groupby(['clusterCellTypes','xenium_to_sc_label']).size()).unstack()
celltype_counts.columns = celltype_counts.columns.droplevel()
celltype_counts.index.name = 'sc cell type'
celltype_counts.columns.name = 'predicted Xenium cell type'
celltype_counts = celltype_counts.T
# Row scale co-occurrence frequencies (by predicted scRNAseq cell type)
celltype_counts = celltype_counts.div(celltype_counts.sum(axis=1), axis=0) 
celltype_counts = celltype_counts.loc[:,celltype_counts.idxmax(axis=0).sort_values().index]
celltype_counts = celltype_counts.fillna(0)
column_sums = celltype_counts.abs().sum(axis=0)
print(column_sums)
# Select columns to keep based on the threshold
columns_to_keep = column_sums[column_sums >= 0.1].index
print(columns_to_keep)
celltype_counts2 = celltype_counts[columns_to_keep]
print(celltype_counts2)
plt.figure(figsize = (14,6))
sns.heatmap(celltype_counts2, cmap='YlGnBu')

In [ ]:
merged_sc.obs.index = merged_sc.obs.index.str.replace("-scSeq", "", regex=False)

In [ ]:
#Reindex sc dataset based on the merged file
adata_Wil_GEpi.obs["xenium_to_sc_label"] = merged_sc.obs["xenium_to_sc_label"].reindex(adata_Wil_GEpi.obs.index)

In [ ]:
#Only keep healthy gingival sections
adata_Wil_HGEpi = adata_Wil_GEpi[adata_Wil_GEpi.obs['status'] == 'H'].copy()

In [ ]:
#Open tabula sapiens epithelial dataset
adata_ts = sc.read_h5ad("/data/vasileiosionat2/Xenium/tabula_sapiens_epithelial/676b8605-7e5f-42c3-940a-aa7042c50f63.h5ad")
#Create "cell_type_integration" column in both the ts and the sc datasets
adata_ts.obs["cell_type_integration"] = adata_ts.obs["tissue"].astype(str) + "_" + adata_ts.obs["cell_type"].astype(str)
adata_Wil_HGEpi.obs["cell_type_integration"] = adata_Wil_HGEpi.obs["tissue"].astype(str) + "_" + adata_Wil_HGEpi.obs["xenium_to_sc_label"].astype(str)
adata_ts.obs['cell_type_integration'].unique().tolist()
# First, check that the two AnnData objects have a common index in their 'obs' columns
common_obs_columns = adata_ts.obs.columns.intersection(adata_Wil_HGEpi.obs.columns)
common_obs_columns
#
adata_ts.var.index = adata_ts.var.index.astype(str)
adata_Wil_HGEpi.var.index = adata_Wil_HGEpi.var.index.astype(str)

In [ ]:
adata_ts.var.index = adata_ts.var.index.astype(str)
adata_Wil_HGEpi.var.index = adata_Wil_HGEpi.var.index.astype(str)

In [ ]:
adata_Wil_HGEpi.var

In [ ]:
#Concatenate tabula sapiens and Xenium-informed gingival single cell dataset after careful readjustment of the adata.var
# Step 1: Standardize column names
adata_ts.var.columns = adata_ts.var.columns.str.lower()
adata_Wil_HGEpi.var.columns = adata_Wil_HGEpi.var.columns.str.lower()

# Step 2: If adata_Wil_HGEpi.var has only one column, rename it
if adata_Wil_HGEpi.var.shape[1] == 1:
    adata_Wil_HGEpi.var.rename(columns={adata_Wil_HGEpi.var.columns[0]: "feature_name"}, inplace=True)  # Adjust if necessary

# Step 3: Align var_names (index) based on Ensembl IDs or Gene Names
# If `adata_ts.var` contains Ensembl IDs, ensure `adata_Wil_HGEpi` matches
if "feature_name" in adata_ts.var.columns:
    adata_ts.var.set_index("feature_name", inplace=True)
    
if "feature_name" in adata_Wil_HGEpi.var.columns:
    adata_Wil_HGEpi.var.set_index("feature_name", inplace=True)

# Step 4: Identify common genes (after renaming/indexing)
common_genes = adata_ts.var_names.intersection(adata_Wil_HGEpi.var_names)

# Step 5: Subset both datasets to only include shared genes
adata_ts = adata_ts[:, common_genes].copy()
adata_Wil_HGEpi = adata_Wil_HGEpi[:, common_genes].copy()

# Step 6: Merge metadata from `adata_ts.var` into `adata_Wil_HGEpi.var`
adata_Wil_HGEpi.var = adata_ts.var.loc[common_genes] 

# Step 7: Concatenate
adata_combined = adata_ts.concatenate(adata_Wil_HGEpi, join='inner', batch_key="batch")

# Step 8: Restore original `var` structure
adata_combined.var = adata_ts.var  # Retain original metadata from `adata1`

In [ ]:
#Normalize, log-transform combined dataset, subset to the top 2000 genes and scale.
sc.pp.normalize_total(adata_combined, target_sum=1e4)  # Normalize counts per cell
sc.pp.log1p(adata_combined)  # Log-transform
sc.pp.highly_variable_genes(adata_combined, flavor='seurat', n_top_genes=2000)
adata_combined = adata_combined[:, adata_combined.var['highly_variable']].copy()
sc.pp.scale(adata_combined)

In [ ]:
#Run pca
sc.tl.pca(adata_combined)

In [ ]:
# This step takes time; ~10 min per iteration
sc.external.pp.harmony_integrate(adata_combined, key="batch", max_iter_harmony=20, max_iter_kmeans=30)

In [ ]:
# Visualize merged with UMAP embedding; takes a long time
sc.pp.neighbors(adata_combined, n_neighbors=30, use_rep="X_pca_harmony", metric="correlation")
sc.tl.umap(adata_combined, min_dist=0.8)

In [ ]:
sc.pl.umap(adata_combined, color='tissue_in_publication')

In [ ]:
adata_combined.obs['tissue_in_publication'] = adata_combined.obs['tissue_in_publication'].cat.add_categories('Gingiva-TAE')
adata_combined.obs['tissue_in_publication'] = adata_combined.obs['tissue_in_publication'].cat.add_categories('Gingiva-OE')

# Now assign the value 'gingiva-OE' to the 'tissue_in_publication' column where 'batch' is 1
adata_combined.obs.loc[adata_combined.obs['batch'] == "1", 'tissue_in_publication'] = 'gingiva_Ep-OE'
# Now assign the value 'gingiva-TAW' to the 'tissue_in_publication' column where 'cell_type_integration' is 1
adata_combined.obs.loc[adata_combined.obs['cell_type_integration'] == "gingiva_Ep.TA", 'tissue_in_publication'] = 'Gingiva-TAE'

In [ ]:
#Subset combined dataset to only include similar barrier tissues
adata_combined_barrier = adata_combined[adata_combined.obs['tissue_in_publication'].isin(["Lung", "Large_Intestine", "Skin",
                                                                                          "Small_Intestine", "Tongue", "Gingiva-OE", "Gingiva-TAE"])].copy()

In [ ]:
#Subset dataset to only include relevant epithelial subsets and exclude secretory epithelial cells or neuroendocrine cells
adata_combined_barrier_Epi = adata_combined_barrier[adata_combined_barrier.obs['cell_type_integration'].isin(['lung_respiratory goblet cell',
 'lung_basal cell',
 'lung_pulmonary alveolar type 2 cell',
 'lung_pulmonary alveolar type 1 cell',
 'skin of chest_epithelial cell',
 'skin of abdomen_epithelial cell',
 'lung_lung ciliated cell',
 'lung_club cell',
 'posterior part of tongue_basal cell',
 'anterior part of tongue_basal cell',
 'anterior part of tongue_stratified squamous epithelial cell',
 'large intestine_enterocyte of epithelium of large intestine',
 'large intestine_paneth cell of colon',
 'large intestine_intestinal crypt stem cell of colon',
 'large intestine_tuft cell of colon',
 'small intestine_enterocyte of epithelium proper of small intestine',
 'small intestine_paneth cell of epithelium of small intestine',
 'small intestine_intestinal crypt stem cell of small intestine',
 'small intestine_small intestine goblet cell',
 'small intestine_intestinal tuft cell',
 'tongue_basal cell',
 'tongue_stratified squamous epithelial cell',
 'large intestine_large intestine goblet cell',
 'large intestine_enterochromaffin-like cell', 
 'small intestine_BEST4+ intestinal epithelial cell, human',
 'small intestine_enterocyte of epithelium proper of ileum',
 'small intestine_enterocyte of epithelium proper of duodenum', 
 'small intestine_transit amplifying cell of small intestine', 
 'ascending colon_large intestine goblet cell',
 'ascending colon_intestinal crypt stem cell of colon',
 'ascending colon_tuft cell of colon',
 'ascending colon_enterocyte of epithelium of large intestine',
 'ascending colon_BEST4+ intestinal epithelial cell, human',
 'ascending colon_transit amplifying cell of colon',
 'ascending colon_enterochromaffin-like cell', 'ileum_enterocyte of epithelium proper of ileum',
 'ileum_small intestine goblet cell',
 'ileum_intestinal crypt stem cell of small intestine',
 'ileum_paneth cell of epithelium of small intestine',
 'ileum_intestinal tuft cell',
 'ileum_BEST4+ intestinal epithelial cell, human', 
 'duodenum_enterocyte of epithelium proper of duodenum',
 'duodenum_BEST4+ intestinal epithelial cell, human',
 'duodenum_paneth cell of epithelium of small intestine',
 'duodenum_small intestine goblet cell',
 'duodenum_intestinal crypt stem cell of small intestine', 'duodenum_intestinal tuft cell',
 'duodenum_transit amplifying cell of small intestine',
 'sigmoid colon_large intestine goblet cell',
 'sigmoid colon_intestinal crypt stem cell of colon',
 'sigmoid colon_BEST4+ intestinal epithelial cell, human',
 'sigmoid colon_enterocyte of epithelium of large intestine',
 'sigmoid colon_transit amplifying cell of colon',
 'sigmoid colon_paneth cell of colon',
 'large intestine_transit amplifying cell of colon', 
  'posterior part of tongue_stratified squamous epithelial cell', 'sigmoid colon_enterochromaffin-like cell',
 'sigmoid colon_tuft cell of colon',
 'ascending colon_paneth cell of colon',
 'duodenum_enterocyte of epithelium proper of ileum', 
 'jejunum_enterocyte of epithelium proper of jejunum',
 'jejunum_intestinal crypt stem cell of small intestine',
 'jejunum_intestinal tuft cell',
 'jejunum_paneth cell of epithelium of small intestine',
 'jejunum_BEST4+ intestinal epithelial cell, human',
 'jejunum_small intestine goblet cell', 'gingiva_Ep.TA',
 'gingiva_Ep.or.b-pb',
 'gingiva_Ep.or.sp',
 'gingiva_Ep.or.k'])].copy()

In [ ]:
# Assuming you've already harmonized and performed PCA

# Step 1: Perform PCA if not already done (assuming PCA is required)
# You can skip this if PCA is already available
sc.pp.pca(adata_combined_barrier_Epi, n_comps=20)

# Step 2: Compute the nearest neighbors
sc.pp.neighbors(adata_combined_barrier_Epi, use_rep='X_pca', n_neighbors=15)

# Step 4: Visualize the coarse clusters
sc.pl.umap(adata_combined_barrier_Epi, color=['tissue_in_publication'])

In [ ]:
#Save combined dataset
adata_combined_Epi.write('path_to_combined_dataset.h5ad')